## Target
it is used car price prediction, evaluation metric is rmse.
the mean price of train is 43878.02. use mean price as single model , the kaggle public/private score is 78643/70105. 

## library & load_data 
### library


In [1]:
library(tidyverse)
library(tidymodels)
library(future)
library(textrecipes)

library(bonsai)
library(lightgbm)

library(lubridate)


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.2.0 ──

✔ broom        1.0.7     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.2.1
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.1.0
✔ parsnip      1.3.0     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt


### loading data


In [2]:
data_path <- '../input/playground-series-s4e9/'
train<- read.csv(file.path(data_path, 'train.csv'))
test <-  read.csv(file.path(data_path, 'test.csv'))
submission <-  read.csv(file.path(data_path, 'sample_submission.csv'))



### quick skim


In [ ]:
# train|> skimr::skim()



── Data Summary ────────────────────────
                           Values
Name                       train 
Number of rows             188533
Number of columns          13    
_______________________          
Column type frequency:           
  character                9     
  numeric                  4     
________________________         
Group variables            None  

── Variable type: character ────────────────────────────────────────────────────
  skim_variable n_missing complete_rate min max empty n_unique whitespace
1 brand                 0             1   3  13     0       57          0
2 model                 0             1   2  55     0     1897          0
3 fuel_type             0             1   0  14  5083        8          0
4 engine                0             1   1  69     0     1117          0
5 transmission          0             1   1  50     0       52          0
6 ext_col               0             1   1  39     0      319          0
7 int_col          

ERROR: Error in is.null(text_repr) || nchar(text_repr) == 0L: 'length = 17' in coercion to 'logical(1)'


In [ ]:
# test|> skimr::skim()



── Data Summary ────────────────────────
                           Values
Name                       test  
Number of rows             125690
Number of columns          12    
_______________________          
Column type frequency:           
  character                9     
  numeric                  3     
________________________         
Group variables            None  

── Variable type: character ────────────────────────────────────────────────────
  skim_variable n_missing complete_rate min max empty n_unique whitespace
1 brand                 0             1   3  13     0       55          0
2 model                 0             1   2  55     0     1891          0
3 fuel_type             0             1   0  14  3383        8          0
4 engine                0             1   1  69     0     1117          0
5 transmission          0             1   1  50     0       52          0
6 ext_col               0             1   1  39     0      317          0
7 int_col          

ERROR: Error in is.null(text_repr) || nchar(text_repr) == 0L: 'length = 17' in coercion to 'logical(1)'


In [ ]:
# submission |> skimr::skim()



── Data Summary ────────────────────────
                           Values    
Name                       submission
Number of rows             125690    
Number of columns          2         
_______________________              
Column type frequency:               
  numeric                  2         
________________________             
Group variables            None      

── Variable type: numeric ──────────────────────────────────────────────────────
  skim_variable n_missing complete_rate    mean     sd      p0     p25     p50
1 id                    0             1 251378. 36284. 188533  219955. 251378.
2 price                 0             1  43878.     0   43878.  43878.  43878.
      p75    p100 hist 
1 282800. 314222  ▇▇▇▇▇
2  43878.  43878. ▁▁▇▁▁


ERROR: Error in is.null(text_repr) || nchar(text_repr) == 0L: 'length = 12' in coercion to 'logical(1)'



### check if train & test is same distribution


In [7]:
get_df_var<-function(df){
  df|>
    summarize_all(var)|>
    pivot_longer(cols=everything(),
                 names_to='feature',
                 values_to='variance')
  
}
train|>get_df_var()


Warning message:
"There were 9 warnings in `summarise()`.
The first warning was:
ℹ In argument: `brand = (function (x, y = NULL, na.rm = FALSE, use) ...`.
Caused by warning:
! NAs introduced by coercion
ℹ Run `dplyr::last_dplyr_warnings()` to see the 8 remaining warnings."


feature,variance
<chr>,<dbl>
id,2.962073e+09
brand,NA
model,NA
model_year,3.204655e+01
milage,2.479857e+09
fuel_type,NA
engine,NA
transmission,NA
ext_col,NA


In [ ]:
get_var_compare <- function(rcp,is_debug=F){
  
  train_var <- rcp|> 
    prep()|>bake(new_data=train)|>
    select(-loan_status, -id) |> # 去除目标变量和ID变量
    summarise_all(var)|>
    pivot_longer(cols = everything(),
               names_to = "feature",
               values_to = "variance")
    
  test_var  <- rcp|>
    prep()|>bake(new_data=test)|>
    select( -id) |> # 去除目标变量和ID变量
    summarise_all(var)|>
    pivot_longer(cols = everything(),
               names_to = "feature",
               values_to = "variance")
  
  compared_result<-
    train_var |> 
    left_join(test_var, by='feature')|>
    mutate(variance_ratio = round(variance.x / variance.y,2))
  
  nfeature_var_changed <- 
    compared_result|>
    filter(variance_ratio >1*1.5 ) |>
    nrow()
 
  if (is_debug){
    print(compared_result)
    
  } 
  return(nfeature_var_changed)
}


## coding
### 1. Data Loading and Initial Exploration ----



### 2. Feature Engineering ----
- leave it in the preprocessing recipe

### 3. Data Splitting ----
#### augment_df


In [ ]:
get_brand_meta <- function(){
  meta_df <- train|>
    summarize(cnt=n(), 
              avg_price=mean(price),
              median_price=median(price),
              min_p=min(price),
              max_p=max(price),
              sd_p=sd(price),
              iqr_p = IQR(price),
              mad_p = mad(price),
              .by = brand)
  return(meta_df)
}
get_engine_meta <- function(){}
get_brand_meta()


#### split/cv



In [ ]:
set.seed(1234)
df_split <- initial_split(train, prop = 0.8, strata = price)
 train_set <- training(df_split)
 test_set <- testing(df_split)
cv_folds <- vfold_cv(train_set,v = 5,strata=price)


### 4. Preprocessing Recipe ----

#### 4.1 v0 base_line


In [ ]:
rcp_bs_v0 <-
  recipe(price ~ ., data = train) |>
  update_role(id, new_role = "id") |>
  step_rm(ext_col,int_col)|>
  step_impute_median(all_numeric_predictors())|>
  step_mutate(model_year=2025-model_year,
              milage_year=milage/model_year,
              accident=case_when(
                accident=='None reported'~'none',
                accident=='At least 1 accident or damage reported'~'report',
                accident==''~NA,
                .default='unknow'))|>
  step_mutate(brand= as.factor(brand),
              model= as.factor(model),
              engine= as.factor(engine),
              #ext_col= as.factor(ext_col),
              #int_col= as.factor(int_col),
              transmission= as.factor(transmission),
              clean_title= as.factor(clean_title),
              accident= as.factor(accident),
              fuel_type=as.factor(fuel_type))|>
  step_novel(all_nominal_predictors())|>
  step_unknown(all_nominal_predictors())|>
  step_other(all_nominal_predictors(),threshold=0.01)|>
  step_dummy(all_nominal_predictors(),one_hot = TRUE)|>
  
  #step_log(price, offset=1, skip=TRUE) |>
  step_log(milage,model_year,milage_year ,offset=1) |>
  
  step_nzv(all_numeric_predictors())|>
  step_normalize(all_numeric_predictors())|>
  step_corr(all_predictors())|>
  check_missing(all_predictors())
 
#rcp_bs_v0 |>prep()
  


#### 4.2 v1 base_line with engine feature


In [ ]:
rcp_bs_v1 <- 
   recipe(price ~ ., data = train) |>
  update_role(id, new_role = "id") |>
  step_rm(ext_col,int_col)|>
  step_mutate(
    hp =stringr::str_extract(engine, "^(\\d+\\.?\\d*)HP|^(\\d+)HP")|>readr::parse_number(),
    hp_missing = case_when(is.na(hp)~1,.default=0),
    litre = stringr::str_extract(engine, "(\\d+\\.?\\d*)L|(\\d+\\.?\\d*) Liter")|>readr::parse_number(),
    litre_missing = case_when(is.na(litre)~1,.default=0),
    engine_type = stringr::str_extract(engine,'Rotary|H6|H4|V6|V-8|V8|V10|V12|W12|I3|I4|I6|\\s[4|5|6|8]\\s| 10 | 12 '),
    is_ecar=case_when(grepl('Electric',engine)~1,.default=0),
    voltage = stringr::str_extract(engine, " (\\d\\d)V")|>readr::parse_number(),
    voltage_missing = case_when(is.na(voltage)~1,.default=0),
    gdi_type = stringr::str_extract(engine, "GDI|PFI|EFI|MPFI|DDI|SIDI"),
    gas_position = stringr::str_extract(engine, "OHV|OHC|SOHC|DOHC"),
    cylinder = stringr::str_extract(engine, "(\\d+) Cylinder|(\\d+) Cylinder Engine|(\\d+)(?=[^\\d]*Cylinder)"),
    is_turbo = case_when(grepl('Turbo|turbo',engine)~'YES'),
    is_twinturbo= case_when(grepl('Twin|twin',engine)~'YES'),
    fuel_info = stringr::str_extract(engine, "Gasoline|Diesel|Electric|Flexible Fuel|Gasoline/Mild Electric Hybrid|Flex Fuel Capability") )|>
  step_rm(engine)|>
   step_impute_median(all_numeric_predictors())|>
  step_mutate(model_year=2025-model_year,
              milage_year=milage/model_year,
              accident=case_when(
                accident=='None reported'~'none',
                accident=='At least 1 accident or damage reported'~'report',
                accident==''~NA,
                .default='unknow'))|>
  step_mutate(brand= as.factor(brand),
              model= as.factor(model),
              #engine= as.factor(engine),
              #ext_col= as.factor(ext_col),
              #int_col= as.factor(int_col),
              transmission= as.factor(transmission),
              clean_title= as.factor(clean_title),
              accident= as.factor(accident),
              fuel_type=as.factor(fuel_type))|>
   step_novel(all_nominal_predictors())|>
  step_unknown(all_nominal_predictors())|>
  step_other(all_nominal_predictors(),threshold=0.01)|>
 step_dummy(all_nominal_predictors(),one_hot = TRUE)|>
 #step_log(price, offset=1, skip=TRUE) |> # lead to bad rmse, remove it
 step_log(milage,model_year,milage_year ,offset=1) |>
 step_nzv(all_numeric_predictors())|>
 step_normalize(all_numeric_predictors())|>
 step_corr(all_predictors())|>
 check_missing(all_predictors())
# rcp_bs_v1 |>prep() |>bake(new_data = train_set)



#### 4.3 v2 base_line with transmission feature


In [ ]:
rcp_bs_v2 <- 
   recipe(price ~ ., data = train) |>
  update_role(id, new_role = "id") |>
  step_rm(ext_col,int_col)|>
  step_mutate(speed =stringr::str_extract(transmission, "^(\\d+)-Speed")|>readr::parse_number(),
              speed_missing = case_when(is.na(speed)~1,.default=0), 
              trans_mode=stringr::str_extract(transmission, 'Dual Shift Mode|Auto-Shift|A/T|AT|M/T|Mt|Automatic|At/Mt|F|Manual|Overdrive|CVT|Variable')) |>
  step_rm(engine)|>
  step_rm(transmission)|>
  step_impute_median(all_numeric_predictors())|>
  step_mutate(model_year=2025-model_year,
              milage_year=milage/model_year,
              accident=case_when(
                accident=='None reported'~'none',
                accident=='At least 1 accident or damage reported'~'report',
                accident==''~NA,
                .default='unknow'))|>
  step_mutate(brand= as.factor(brand),
              model= as.factor(model),
              #engine= as.factor(engine),
              #ext_col= as.factor(ext_col),
              #int_col= as.factor(int_col),
              #transmission= as.factor(transmission),
              clean_title= as.factor(clean_title),
              accident= as.factor(accident),
              fuel_type=as.factor(fuel_type))|>
   step_novel(all_nominal_predictors())|>
  step_unknown(all_nominal_predictors())|>
  step_other(all_nominal_predictors(),threshold=0.01)|>
  step_dummy(all_nominal_predictors(),one_hot = TRUE)|>
  #step_log(price, offset=1, skip=TRUE) |> # lead to bad rmse, remove it
  step_log(milage,model_year,milage_year ,offset=1) |>
  step_nzv(all_numeric_predictors())|>
  step_normalize(all_numeric_predictors())|>
  step_corr(all_numeric_predictors())|>
  check_missing(all_predictors())
#rcp_bs_v2 |> prep()|> bake(new_data = train_set)



#### 4.3 v3 base_line with model


In [ ]:
rcp_bs_v3<-recipe(price ~ ., data = train) |>
  update_role(id, new_role = "id") |>
  step_rm(ext_col,int_col)|>
  step_mutate(model_type=stringr::str_extract(tolower(model),
                                '([a-z]-class)|(premium|luxury|base|sport|premium plus|premium|turbo|matic|se|sel|platinum|limited|deluxe|laramie|lux|luxury|ultimate|gt|hse|xle|xlt)|( lt)'),
         model_word = stringr::str_extract(model, "^\\b[[:alnum:]-]+\\b"),
         # Extract the first number
         model_number = stringr::str_extract(stringr::str_replace(model, "\\b[[:alnum:]-]+\\b", ""), "\\d+"))|>
  step_rm(model)|>
  step_impute_median(all_numeric_predictors())|>
  step_mutate(model_year=2025-model_year,
              milage_year=milage/model_year,
              accident=case_when(
                accident=='None reported'~'none',
                accident=='At least 1 accident or damage reported'~'report',
                accident==''~NA,
                .default='unknow'))|>
  step_mutate(brand= as.factor(brand),
              #model= as.factor(model),
              engine= as.factor(engine),
              #ext_col= as.factor(ext_col),
              #int_col= as.factor(int_col),
              transmission= as.factor(transmission),
              clean_title= as.factor(clean_title),
              accident= as.factor(accident),
              fuel_type=as.factor(fuel_type))|>
  step_novel(all_nominal_predictors())|>
  step_unknown(all_nominal_predictors())|>
  step_other(all_nominal_predictors(),threshold=0.01)|>
  step_dummy(all_nominal_predictors(),one_hot = TRUE)|>
  
  #step_log(price, offset=1, skip=TRUE) |>
  step_log(milage,model_year,milage_year ,offset=1) |>
  
  step_nzv(all_numeric_predictors())|>
  step_normalize(all_numeric_predictors())|>
  step_corr(all_predictors())|>
  check_missing(all_predictors())
 
#rcp_bs_v3|>prep()|>juice()|>glimpse()



#### 4.4 v4 base_line with engine & transmission


In [ ]:
rcp_bs_v4 <- 
   recipe(price ~ ., data = train) |>
  update_role(id, new_role = "id") |>
  step_rm(ext_col,int_col)|>
   step_mutate(
    hp =stringr::str_extract(engine, "^(\\d+\\.?\\d*)HP|^(\\d+)HP")|>readr::parse_number(),
    hp_missing = case_when(is.na(hp)~1,.default=0),
    litre = stringr::str_extract(engine, "(\\d+\\.?\\d*)L|(\\d+\\.?\\d*) Liter")|>readr::parse_number(),
    litre_missing = case_when(is.na(litre)~1,.default=0),
    engine_type = stringr::str_extract(engine,'Rotary|H6|H4|V6|V-8|V8|V10|V12|W12|I3|I4|I6|\\s[4|5|6|8]\\s| 10 | 12 '),
    is_ecar=case_when(grepl('Electric',engine)~1,.default=0),
    voltage = stringr::str_extract(engine, " (\\d\\d)V")|>readr::parse_number(),
    voltage_missing = case_when(is.na(voltage)~1,.default=0),
    gdi_type = stringr::str_extract(engine, "GDI|PFI|EFI|MPFI|DDI|SIDI"),
    gas_position = stringr::str_extract(engine, "OHV|OHC|SOHC|DOHC"),
    cylinder = stringr::str_extract(engine, "(\\d+) Cylinder|(\\d+) Cylinder Engine|(\\d+)(?=[^\\d]*Cylinder)"),
    is_turbo = case_when(grepl('Turbo|turbo',engine)~'YES'),
    is_twinturbo= case_when(grepl('Twin|twin',engine)~'YES'),
    fuel_info = stringr::str_extract(engine, "Gasoline|Diesel|Electric|Flexible Fuel|Gasoline/Mild Electric Hybrid|Flex Fuel Capability") )|>
    step_mutate(speed =stringr::str_extract(transmission, "^(\\d+)-Speed")|>readr::parse_number(),
              speed_missing = case_when(is.na(speed)~1,.default=0), 
              trans_mode=stringr::str_extract(transmission, 'Dual Shift Mode|Auto-Shift|A/T|AT|M/T|Mt|Automatic|At/Mt|F|Manual|Overdrive|CVT|Variable')) |>
  step_rm(engine)|>
  step_rm(transmission)|>
  step_impute_median(all_numeric_predictors())|>
  step_mutate(model_year=2025-model_year,
              milage_year=milage/model_year,
              accident=case_when(
                accident=='None reported'~'none',
                accident=='At least 1 accident or damage reported'~'report',
                accident==''~NA,
                .default='unknow'))|>
  step_mutate(brand= as.factor(brand),
              model= as.factor(model),
              #engine= as.factor(engine),
              #ext_col= as.factor(ext_col),
              #int_col= as.factor(int_col),
              #transmission= as.factor(transmission),
              clean_title= as.factor(clean_title),
              accident= as.factor(accident),
              fuel_type=as.factor(fuel_type))|>
   step_novel(all_nominal_predictors())|>
  step_unknown(all_nominal_predictors())|>
  step_other(all_nominal_predictors(),threshold=0.01)|>
  step_dummy(all_nominal_predictors(),one_hot = TRUE)|>
  #step_log(price, offset=1, skip=TRUE) |> # lead to bad rmse, remove it
  step_log(milage,model_year,milage_year ,offset=1) |>
  
  step_nzv(all_numeric_predictors())|>
  step_normalize(all_numeric_predictors())|>
  step_corr(all_predictors())|>
  check_missing(all_predictors())



#### 4.5 v1+v3 engine + model 


In [ ]:
rcp_bs_v5 <- 
   recipe(price ~ ., data = train) |>
  update_role(id, new_role = "id") |>
  step_rm(ext_col,int_col)|>
  step_mutate(
    hp =stringr::str_extract(engine, "^(\\d+\\.?\\d*)HP|^(\\d+)HP")|>readr::parse_number(),
    hp_missing = case_when(is.na(hp)~1,.default=0),
    litre = stringr::str_extract(engine, "(\\d+\\.?\\d*)L|(\\d+\\.?\\d*) Liter")|>readr::parse_number(),
    litre_missing = case_when(is.na(litre)~1,.default=0),
    engine_type = stringr::str_extract(engine,'Rotary|H6|H4|V6|V-8|V8|V10|V12|W12|I3|I4|I6|\\s[4|5|6|8]\\s| 10 | 12 '),
    is_ecar=case_when(grepl('Electric',engine)~1,.default=0),
    voltage = stringr::str_extract(engine, " (\\d\\d)V")|>readr::parse_number(),
    voltage_missing = case_when(is.na(voltage)~1,.default=0),
    gdi_type = stringr::str_extract(engine, "GDI|PFI|EFI|MPFI|DDI|SIDI"),
    gas_position = stringr::str_extract(engine, "OHV|OHC|SOHC|DOHC"),
    cylinder = stringr::str_extract(engine, "(\\d+) Cylinder|(\\d+) Cylinder Engine|(\\d+)(?=[^\\d]*Cylinder)"),
    is_turbo = case_when(grepl('Turbo|turbo',engine)~'YES'),
    is_twinturbo= case_when(grepl('Twin|twin',engine)~'YES'),
    fuel_info = stringr::str_extract(engine, "Gasoline|Diesel|Electric|Flexible Fuel|Gasoline/Mild Electric Hybrid|Flex Fuel Capability") )|>
   step_mutate(model_type=stringr::str_extract(tolower(model),
                                '([a-z]-class)|(premium|luxury|base|sport|premium plus|premium|turbo|matic|se|sel|platinum|limited|deluxe|laramie|lux|luxury|ultimate|gt|hse|xle|xlt)|( lt)'),
         model_word = stringr::str_extract(model, "^\\b[[:alnum:]-]+\\b"),
         # Extract the first number
         model_number = stringr::str_extract(stringr::str_replace(model, "\\b[[:alnum:]-]+\\b", ""), "\\d+"))|>
  step_rm(model)|>
  step_rm(engine)|>
   step_impute_median(all_numeric_predictors())|>
  step_mutate(model_year=2025-model_year,
              milage_year=milage/model_year,
              accident=case_when(
                accident=='None reported'~'none',
                accident=='At least 1 accident or damage reported'~'report',
                accident==''~NA,
                .default='unknow'))|>
  step_mutate(brand= as.factor(brand),
              #model= as.factor(model),
              #engine= as.factor(engine),
              #ext_col= as.factor(ext_col),
              #int_col= as.factor(int_col),
              transmission= as.factor(transmission),
              clean_title= as.factor(clean_title),
              accident= as.factor(accident),
              fuel_type=as.factor(fuel_type))|>
   step_novel(all_nominal_predictors())|>
  step_unknown(all_nominal_predictors())|>
  step_other(all_nominal_predictors(),threshold=0.01)|>
 step_dummy(all_nominal_predictors(),one_hot = TRUE)|>
 #step_log(price, offset=1, skip=TRUE) |> # lead to bad rmse, remove it
 step_log(milage,model_year,milage_year ,offset=1) |>
 step_nzv(all_numeric_predictors())|>
 step_normalize(all_numeric_predictors())|>
 step_corr(all_predictors())|>
 check_missing(all_predictors())


 
#### all recipes


In [ ]:
set.seed(1234)
library(future)
library(furrr)
selected_rcps <- list(#base=rcp_bs_v0,
                      v1_engine=rcp_bs_v1,
                      #v2_transmission=rcp_bs_v2,
                      #v3_model = rcp_bs_v3,
                      #v4_eng_trans=rcp_bs_v4,
                      v5_engin_model= rcp_bs_v5)
plan(multisession,workers = 5)
selected_rcps|>future_map(\(rcp_item) rcp_item|>prep()|>bake(new_data=train)|>summary())
plan(sequential)



### 5. Model Specification ----


In [ ]:
lgbm_eng<-
   boost_tree(
     # trees = 500, # Number of trees
     # learn_rate = 0.1,
     # loss_reduction = 0.001,
     # sample_size = 0.85, # Added sample_size
     # tree_depth = tune(),
     # mtry = tune(),
     # min_n = tune()
   ) |>
   set_mode("regression")|>
   set_engine("lightgbm",
              metric='rmse', 
              # num_leaves = 30,
              num_threads = 4,
              verbose=1) 

lm_eng<-
   linear_reg() |>
   set_mode("regression")|>
   set_engine("lm") 

selected_eng <- list(linear=lm_eng,
                     lgbm=lgbm_eng)


### 6. Workflow ----
#### simple wflow


In [ ]:
set.seed(1234)
simple_wf_fit <-
  workflow() |>
  add_recipe(rcp_bs_v1) |>
  add_model(lgbm_eng)|>
  last_fit(df_split)

simple_wf_fit|>collect_metrics()
# simple_wf_fit |>
#   extract_fit_engine()|>
#   plot()

simple_wf_diag_df<- 
  simple_wf_fit|>
  collect_predictions()|>
  mutate(.residual=price-.pred)|>
  select(.row,.pred,.residual)|>
  bind_cols(test_set)|>
  relocate(all_of(c('price','.pred','.residual')),.after=1)


#### simple workflowset



In [ ]:
set.seed(1234)
library(future)
plan(multisession,workers = 12)
ctrl <- control_resamples(save_pred = TRUE, save_workflow = TRUE,verbose=TRUE)
wfs_result <-
  workflow_set(preproc = selected_rcps,
               models = selected_eng ) |>
  workflow_map(fn='fit_resamples',
               resamples =cv_folds,
               metrics = metric_set(rmse, rsq),
               control = ctrl
               )
wfs_result|>collect_metrics()  
  
plan(sequential)


### 7 stacking


In [ ]:
combined_fit <-
  stacks::stacks()|>
  stacks::add_candidates(wfs_result)|>
  stacks::blend_predictions()|>
  stacks::fit_members()

combined_fit|>
  autoplot(type = "weights")

autoplot(combined_fit)



### 7. Tuning Grid ----


In [ ]:
# cars_grid <- grid_space_filling(
#   #learn_rate(range = c(0.01, 0.1)),
#   # loss_reduction(range = c(0, 10)), #Keep if you want it.
#   #bag_fraction(range = c(0.7, 0.9)), # Specify prop = TRUE.
#   tree_depth(range = c(5, 10)),
#   finalize(mtry(range = c(2, 10)),
#            select(cars_train_set, -Price)),
#   min_n(range = c(2, 20)),
#   size = 10
# )



### 8. Cross-Validation ----


In [ ]:
# combined it with step3 data splitting




### 9. Tuning and Evaluation ----


In [ ]:
# plan(multisession,workers =2)
# cars_tune_results <- cars_workflow |>
#   tune_grid(
#     resamples = cars_folds,
#     grid = cars_grid,
#     metrics = metric_set(rmse),
#      control = control_grid(save_pred = TRUE, 
#                             verbose = TRUE,
#                             allow_par = F) # Keep predictions
#   )
#  
#  # Find best parameters
#  best_params <- cars_tune_results |>
#    select_best("rmse")
# 
#  # Finalize workflow with best parameters
#  final_workflow <- cars_workflow |>
#    finalize_workflow(best_params)


In [ ]:
# Fit the final workflow to the training data
# final_lgbm_fit <- last_fit(final_workflow,cars_split )
# final_lgbm_mod <- extract_workflow(final_lgbm_fit )
# collect_metrics(final_lgmb_mod)

# plan(sequential)



### 10. Evaluate on Test Set ----


In [ ]:
combined_test_result <- 
  test_set %>%
  bind_cols(predict(combined_fit, .))
combined_test_result|>rsq(price, .pred)



### 11. Prepare Submission ----


In [ ]:
final_model <- combined_fit#simple_wf_fit|>extract_workflow()
final_predictions <- final_model |>
   predict(new_data = test) 

 #Handle negative predictions
 final_predictions <- final_predictions |>
   mutate(.pred= ifelse(.pred< 0, 0, .pred))

 # Save submission file
 submission |>
   mutate(price=final_predictions$.pred)|>
   readr::write_csv("submission.csv")
 zip('submission.csv.zip','submission.csv')
 



## kaggle submission


In [ ]:
# submit latest submission.csv
system('kaggle competitions submit -c playground-series-s4e9 -f submission.csv.zip -m "stacking 69827/15.8% "')
Sys.sleep(15)
# get latest score 
system('kaggle competitions submissions -q -c playground-series-s4e9')

# get leader board score
#system('kaggle competitions leaderboard -s -v -c playground-series-s4e9')
